In [2]:
import tensorflow as tf
import numpy as np
import math
import os
import random
import numpy as np
import pandas as pd
from pandas import DataFrame
import cPickle as pickle
import math
import copy

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [4]:
df_train = pd.read_csv('../data/rl_train_set_unscaled.csv')
df_val = pd.read_csv('../data/rl_val_set_unscaled.csv')
df_test = pd.read_csv('../data/rl_test_set_unscaled.csv')

In [5]:
# let the mortality labels  be -1 and 1: -1 for survival
df_train['died_in_hosp'][df_train['died_in_hosp'] == 0] = -1
df_val['died_in_hosp'][df_val['died_in_hosp'] == 0] = -1
df_test['died_in_hosp'][df_test['died_in_hosp'] == 0] = -1

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [6]:
target_feat = list(np.loadtxt('../data/state_features_pred.txt', dtype=str))
target_feat.append('died_in_hosp')

cur_feat = list(np.loadtxt('../data/state_features.txt', dtype=str))
cur_feat.append('iv_input')
cur_feat.append('vaso_input')

In [7]:
# define an action mapping - how to get an id representing the action from the (iv,vaso) tuple
action_map = {}
count = 0
for iv in range(5):
    for vaso in range(5):
        action_map[(iv,vaso)] = count
        count += 1

In [8]:
inv_action_map = {}
for iv in range(5):
    for vaso in range(5):
        inv_action_map[5*iv+vaso] = [iv,vaso]    

In [9]:
# Change the actions: {0: -0.8, 1: -0.4, 2: 0, 3: 0.4, 4: 0.8}. This will help for the policy search, which
# uses a continuous action space
lookup ={0:-0.8, 1:-0.4, 2:0, 3:0.4, 4:0.8}
for dfm in (df_train, df_val, df_test):
    for iv_input in range(5):
        val = lookup[iv_input]
        dfm['iv_input'][dfm['iv_input'] == iv_input] = val
    for vaso_input in range(5):
        val = lookup[vaso_input]
        dfm['vaso_input'][dfm['vaso_input'] == vaso_input] = val

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [10]:
# find the longest trajectory
orig_df = pd.read_csv('../data/MKdataset07Feb17.csv')
max_time_steps = max(orig_df['bloc'])
del orig_df

In [11]:
a = np.arange(9).reshape([3,3])
np.pad(a, [[0, 5-a.shape[0]],[0,0]], 'constant', constant_values=0)

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8],
       [0, 0, 0],
       [0, 0, 0]])

In [12]:
# X: [[s1a1, s2a2, ... , sTaT], [...] ] ie, (s,a) tuples for every trajectory in dataset
# Y: [[delta1, delta2, ... , deltaT], [...] ] ie, deltas for every 
# make X of shape: num_traj*max_time_steps*len(cur_feat)
# make Y of shape: num_traj*max_time_steps*len(target_feat)
def make_data(df_in):
    X = []
    Y = []
    seq_lens = []
    traj_ids = df_in['icustayid'].unique()

    for traj_id in traj_ids:
        rel_df = df_in.loc[df_in['icustayid'] == traj_id]
        state = rel_df[cur_feat].values[:-1] # take everything except the final state in the trajectory
        
        tmp1 = rel_df[target_feat].values
        tmp2 = tmp1[1:]
        tmp1 = tmp1[:-1]
        target = tmp2 - tmp1
        # add the mortality labels in
        target[:,-1] = rel_df['died_in_hosp'].values[:-1]
        
        # store the length of the seq for dynamic RNN
        seq_lens.append(len(state))
        
        # do padding if necessary
        if len(state) < max_time_steps:
            state = np.pad(state, [[0, max_time_steps-state.shape[0]],[0,0]],'constant', constant_values=0)
            target = np.pad(target, [[0, max_time_steps-target.shape[0]],[0,0]],'constant', constant_values=0)
        Y.append(target)
        X.append(state)

        if count % 10000 == 0 and count > 0:
            print count

    return np.array(X),np.array(Y), np.array(seq_lens)

In [13]:
dire = 'recurrent_data/'
if not os.path.exists(dire):
    os.makedirs(dire)
    
if not os.path.exists(dire + 'X_train.npy'):
    x_train, y_train, seqlen_train = make_data(df_train)
    np.save(dire + 'X_train.npy',x_train)
    np.save(dire + 'Y_train.npy',y_train)
    np.save(dire + 'seqlen_train.npy',seqlen_train)
    print "Saved train"
else:
    x_train = np.load(dire + 'X_train.npy')
    y_train = np.load(dire + 'Y_train.npy')
    seqlen_train = np.load(dire + 'seqlen_train.npy')
    print "Loaded train"

if not os.path.exists(dire + 'X_val.npy'):
    x_val,y_val, seqlen_val = make_data(df_val)
    np.save(dire + 'X_val.npy',x_val)
    np.save(dire + 'Y_val.npy',y_val)
    np.save(dire + 'seqlen_val.npy',seqlen_val)
    print "Saved val"
else:
    x_val = np.load(dire + 'X_val.npy')
    y_val = np.load(dire + 'Y_val.npy')
    seqlen_val = np.load(dire + 'seqlen_val.npy')
    print "Loaded val"
    
if not os.path.exists(dire + 'X_test.npy'):
    x_test, y_test, seqlen_test = make_data(df_test)
    np.save(dire + 'X_test.npy',x_test)
    np.save(dire +    'Y_test.npy',y_test)
    np.save(dire + 'seqlen_test.npy',seqlen_test)
    print "Saved test"
else:
    x_test = np.load(dire + 'X_test.npy')
    y_test = np.load(dire + 'Y_test.npy')
    seqlen_test = np.load(dire + 'seqlen_test.npy')
    print "Loaded test"

Loaded train
Loaded val
Loaded test


In [14]:
print x_train.shape, y_train.shape
print x_val.shape, y_val.shape
print x_test.shape, y_test.shape

(12528, 20, 50) (12528, 20, 42)
(1789, 20, 50) (1789, 20, 42)
(3581, 20, 50) (3581, 20, 42)


In [15]:
# Sequence LSTM -- predicts change in key variables
hidden1 = 25
hidden2 = 32
# target_size = 1  #just predict sofa
target_size = y_train.shape[2]
class EnvModel():
    def __init__(self):
        self.input = tf.placeholder(tf.float32, shape=[None, max_time_steps, len(cur_feat)])
        self.targets = tf.placeholder(tf.float32, shape=[None, max_time_steps, target_size])
        self.seq_len = tf.placeholder(tf.int32, shape = None)
        self.rnn_layers = [tf.contrib.rnn.LSTMCell(size) for size in [hidden1, target_size]]
        #self.rnn_layers = [tf.contrib.rnn.LSTMCell(size) for size in [hidden1, hidden2, target_size]]
        self.multi_rnn_cell = tf.contrib.rnn.MultiRNNCell(self.rnn_layers)
        self.outputs, self.state = tf.nn.dynamic_rnn(cell=self.multi_rnn_cell,
                                       inputs=self.input,
                                       sequence_length =self.seq_len,
                                       dtype=tf.float32)
        # 'outputs' is a tensor of shape [batch_size, max_time_steps, target_size]
        # 'state' is a N-tuple where N is the number of LSTMCells containing a
        # tf.contrib.rnn.LSTMStateTuple for each cell
        self.loss = tf.reduce_mean(tf.square(self.targets-self.outputs))
        self.reg_lambda = 0.0001

        self.vars = tf.trainable_variables() 
        self.lossL2 = tf.add_n([ tf.nn.l2_loss(v) for v in self.vars
                    if 'bias' not in v.name ]) * self.reg_lambda    

        self.trainer = tf.train.AdamOptimizer(learning_rate=0.001)
        self.update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(self.update_ops):
        # Ensures that we execute the update_ops before performing the model update, so batchnorm works
            self.update_model = self.trainer.minimize(self.loss + self.lossL2)



In [16]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # Don't use all GPUs 
config.allow_soft_placement = True  # Enable manual control

In [17]:
eval_batch = 64
def run_eval(sess, env_mdl):
    start = 0
    end = 0
    net_loss = 0.0
    while start < len(x_val):
        end = min(len(x_val), start+eval_batch)
        x_batch = x_val[start:end]
        y_batch = y_val[start:end]
        seqlen_batch = seqlen_val[start:end]

        # extract out only sofa for target
#         y_batch = y_batch[:,:, 29]
#         y_batch = y_batch[:,:, None]

        loss = sess.run(env_model.loss,
                        feed_dict={env_model.input:x_batch,
                                   env_model.targets:y_batch,
                                   env_model.seq_len:seqlen_batch})
    
        # update net loss
        net_loss += loss

        # increment start index to inds
        start += eval_batch
    
    avg_eval_loss = net_loss/(len(x_val)/float(eval_batch))
    
    return avg_eval_loss
        

In [18]:
batch_size = 64
num_epochs = 20
load_model = False #Whether to load a saved model.
save_dir = "./env_model_rnn/"
save_path = "./env_model_rnn/ckpt"#The path to save our model to.
tf.reset_default_graph()
env_model = EnvModel()

saver = tf.train.Saver(tf.global_variables())

init = tf.global_variables_initializer()

trainables = tf.trainable_variables()

#Make a path for our model to be saved in.
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

with tf.Session(config=config) as sess:
    if load_model == True:
        print('Trying to load model...')
        try:
            restorer = tf.train.import_meta_graph(save_path + '.meta')
            restorer.restore(sess, tf.train.latest_checkpoint(save_dir))
            print "Model restored"
        except IOError:
            print "No previous model found, running default init"
            sess.run(init)
    else:
        print("Running default init")
        sess.run(init)
    print("Init done")
    
    for j in range(1,num_epochs+1):
        net_loss = 0.0
        inds = np.random.permutation(x_train.shape[0])
        start_idx = 0
        end_idx = 0
        while start_idx < len(x_train):
            end_idx = min(len(x_train), start_idx+batch_size)
            batch_inds = inds[start_idx:end_idx]
            x_batch = x_train[batch_inds]
            y_batch = y_train[batch_inds]
            seqlen_batch = seqlen_train[batch_inds]
            # extract out only sofa for target
#             y_batch = y_batch[:,:, 29]
#             y_batch = y_batch[:,:, None]
            
            # train using the batch
            _,loss = sess.run([env_model.update_model,env_model.loss], \
            feed_dict={env_model.input:x_batch,
                       env_model.targets:y_batch,
                       env_model.seq_len:seqlen_batch})
            
            # update net loss
            net_loss += loss
            
            # increment start index to inds
            start_idx += batch_size
        
        saver.save(sess,save_path)
        print("Saved Model, epoch " + str(j))
        
        av_loss = net_loss/(len(x_train)/float(batch_size))
        print("Average training loss per batch is ", av_loss)
        net_loss = 0.0
        
        eval_loss = run_eval(sess, env_model)
        
        print ("Avg val loss at epoch end is " + str(eval_loss))

    

Running default init
Init done
Saved Model, epoch 1
('Average training loss per batch is ', 0.11886537463272212)
Avg val loss at epoch end is 0.111391836735
Saved Model, epoch 2
('Average training loss per batch is ', 0.11037431210089156)
Avg val loss at epoch end is 0.106728300027
Saved Model, epoch 3
('Average training loss per batch is ', 0.10706470999985666)
Avg val loss at epoch end is 0.104728123807
Saved Model, epoch 4
('Average training loss per batch is ', 0.10559951273265317)
Avg val loss at epoch end is 0.1035458891
Saved Model, epoch 5
('Average training loss per batch is ', 0.10475373481273043)
Avg val loss at epoch end is 0.102878025486
Saved Model, epoch 6
('Average training loss per batch is ', 0.1041547190945115)
Avg val loss at epoch end is 0.102443620704
Saved Model, epoch 7
('Average training loss per batch is ', 0.1037721182879817)
Avg val loss at epoch end is 0.102114128627
Saved Model, epoch 8
('Average training loss per batch is ', 0.10344979977699076)
Avg val l

In [59]:
# with tf.Session(config=config) as sess:
#     print('Trying to load model...')
#     try:
#         restorer = tf.train.import_meta_graph(save_path + '.meta')
#         restorer.restore(sess, tf.train.latest_checkpoint(save_dir))
#         print "Model restored"
#     except IOError:
#         print "No previous model found, running default init"
#         sess.run(init)
#     print("Init done")
    
#     bx = x_train[50:100]
#     by = y_train[50:100, 29]
#     by = by[:, None]
#     op = sess.run(env_model.output, \
#             feed_dict={env_model.cur_state:bx,
#                        env_model.targets:by,
#                        env_model.phase:False})

# # print op.shape, by.shape
# # for i in zip(np.squeeze(op), np.squeeze(by)):
# #     print i

Trying to load model...
No previous model found, running default init


NameError: name 'init' is not defined

In [63]:
np.mean((np.squeeze(op)-np.squeeze(by))**2)

0.11017323824480833

In [64]:
for i in zip(np.squeeze(op), np.squeeze(by)):
    print i

(0.17911743, 0.0)
(0.093046457, 0.0)
(0.13156834, 0.28613931514300006)
(0.075183168, 0.0)
(0.15709624, 1.144557260572)
(0.053191449, 0.28613931513999979)
(0.007503923, -0.28613931513999979)
(0.086839095, 0.0)
(0.059295628, 0.0)
(-0.0029163994, 0.0)
(0.02531508, 0.28613931513999979)
(-0.036283854, 0.0)
(-0.039782193, 0.0)
(-0.10121112, -0.57227863027999981)
(-0.16417766, -0.28613931514799995)
(0.13331677, 0.0)
(0.053894948, 0.0)
(0.21118657, 0.28613931514799995)
(0.16026877, 0.0)
(0.082047328, 0.28613931514000002)
(-0.057933476, 0.0)
(0.035227526, 0.28613931513999979)
(-0.036803689, 0.0)
(-0.12285101, -0.28613931513999979)
(-0.14585981, 0.0)
(-0.14774466, -0.28613931514000002)
(-0.19879135, -0.28613931514799995)
(-0.043731753, 0.0)
(0.027128305, 0.0)
(-0.092596322, -1.144557260572)
(0.24658708, 0.0)
(0.16986302, 0.0)
(0.16923501, 0.0)
(0.14478707, 0.0)
(0.09138447, 0.0)
(0.12362672, 0.0)
(0.057311695, 0.0)
(0.064386889, 0.5722786302859999)
(-0.077138633, 1.144557260574)
(-0.51417947, -0